In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from inference_toolbox.parameter import Parameter
from inference_toolbox.model import Model
from inference_toolbox.likelihood import Likelihood
from inference_toolbox.sampler import Sampler
from inference_toolbox.visualiser import Visualiser

from data_processing.normaliser import Normaliser
from box_gridder import BoxGridder
from gaussian_processor import GaussianProcessor


import numpy as np

In [2]:
# # Import and select data.
# all_data = pd.read_csv('data/total_data.csv')

# # Import and select metadata.
# metadata = pd.read_csv('data/data_summary.csv',
#     usecols = ['Experiment', 'Wind_Dir', 'WindSpeed', 'boat.lat', 'boat.lon']
# )

In [3]:

# normaliser = Normaliser(all_data, metadata)

# all_experiments = normaliser.get_experiments_list()

# selected_experiments = np.delete(all_experiments, np.where(all_experiments == 'Control'))

# normalised_data = normaliser.normalise_data(selected_experiments)


In [4]:
# box_gridder = BoxGridder(normalised_data)

# averaged_df = box_gridder.get_averages([200,200,50],False)

# # box_gridder.visualise_average_data(averaged_df)

# # averaged_df

In [5]:
# gp = GaussianProcessor(averaged_df, kernel = 'matern_white', data_norm='sqrt')
# training_data, test_data = gp.train_test_split()
# print(training_data)
# gp.train_gp(training_data)
# gp.test(test_data)
# grid = box_gridder.get_grid([10,10,10])
# gp.predict_from_gp(grid, threeD=True, save=True, log_results=True)
# gp.animate() 

In [6]:
# Get dummy data
dummy_data = pd.read_csv('data/dummy_data.csv')

In [7]:
# training_data, testing_data = train_test_split(averaged_df, test_size=0.2)
training_data, testing_data = train_test_split(dummy_data, test_size=0.2)

In [8]:
# Initialize parameter series
params = pd.Series({},dtype='float64')

# Parameter Assignment
I_y = Parameter(init_val = 0.1, step_select = "gamma", step_size = 0.005, prior_select = "no prior")
I_y.add_prior_param("mu",0.1)
I_y.add_prior_param("sigma",0.1)
params['I_y'] = I_y

I_z = Parameter(init_val = 0.1, step_select = "gamma", step_size = 0.005, prior_select = "no prior")
I_z.add_prior_param("mu",0.1)
I_z.add_prior_param("sigma",0.1)
params['I_z'] = I_z

Q = Parameter(init_val = 3e13, step_select = "gamma", step_size = 1e11, prior_select = "no prior")
Q.add_prior_param("mu",3e13)
Q.add_prior_param("sigma",1e13)
params['Q'] = Q

# Model Assignment
model = Model('GPM_alt_norm')
model.add_model_param("H",10)

# Likelihood function assigmnent
likelihood = Likelihood("gaussian_fixed_sigma")
likelihood.add_likelihood_param("sigma",1e13)
# likelihood.add_likelihood_param("lambda_1",1)
# likelihood.add_likelihood_param("lambda_2",0.05)

# Initialize the sampler
sampler = Sampler(params, model, likelihood, training_data, show_sample_info = True)
hyperparams = sampler.get_hyperparams()

# Sample the parameters
params_samples, acceptance_rate = sampler.sample_all(10000)

params_mean = sampler.get_mean_samples(params_samples)
params_lower = sampler.get_lower_samples(params_samples)
params_upper = sampler.get_upper_samples(params_samples)

TypeError: 'module' object is not callable

In [ ]:
x = np.linspace(0.1, 1200, 81)
y = np.linspace(-1000, 1000, 81)
z = np.linspace(0,200,81)

domain = np.array([x,y,z]).T

params_all = [params_lower, params_mean, params_upper]

visualiser = Visualiser(testing_data, params_all, model, hyperparams, acceptance_rate)
instance = visualiser.visualise_results(domain, save = True)
visualiser.get_traceplot(params_samples, instance, save = True)
visualiser.animate(instance)

c:\Users\Sam\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Sam\anaconda3\lib\site-packages\matplotlib\collections.py:967: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
